> Solve Buckley–Leverett using PINN

<center>$\Large fw(u)=\frac{u^2}{u^2+\frac{(1-u)^2}{M}}$</center>

Using the code by Alexandros Papados available in [github.com/alexpapados/Physics-Informed-Deep-Learning-Solid-and-Fluid-Mechanics](https://github.com/alexpapados/Physics-Informed-Deep-Learning-Solid-and-Fluid-Mechanics/blob/main/Buckley-Leverett-Problem/Buckley-Leverett.py)

In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import time
import scipy.io
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

%matplotlib inline

torch.manual_seed(123456)
np.random.seed(123456)

In [3]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.net = nn.Sequential()                                                 # Define neural network
        self.net.add_module('Linear_layer_1', nn.Linear(2, 30))                    # First linear layer
        self.net.add_module('Tanh_layer_1', nn.Tanh())                             # First activation Layer

        for num in range(2, 7):                                                    # Number of layers (2 through 7)
            self.net.add_module('Linear_layer_%d' % (num), nn.Linear(30, 30))      # Linear layer
            self.net.add_module('Tanh_layer_%d' % (num), nn.Tanh())                # Activation Layer
        self.net.add_module('Linear_layer_final', nn.Linear(30, 1))                # Output Layer

    def forward(self, x):
        return self.net(x)

    # Loss function for PDE
    def loss_pde(self, x):
        u = self.net(x)
        # Gradients and partial derivatives
        du_g = gradients(u, x)[0]                                  # Gradient [u_t, u_x]
        u_t, u_x = du_g[:, :1], du_g[:, 1:]                        # Partial derivatives u_t, u_x
        F = (u**2)/(4*(u**2) + (1-u)**2)
        DF = gradients(F, x)[0]                                    # Gradient of flux DF
        F_x = DF[:, 1:]                                            # Partial derivativEe of flux, F(u)_x

        # Loss function for the Euler Equations
        f = ((u_t + F_x)**2).mean()
        return f

    # Loss function for initial condition
    def loss_ic(self, x_ic, u_ic):
        y_ic = self.net(x_ic)                                      # Initial condition
        u_ic_nn = y_ic[:, 0]

        # Loss function for the initial condition
        loss_ics = ((u_ic_nn - u_ic) ** 2).mean()
        return loss_ics

In [4]:
# Calculate gradients using torch.autograd.grad
def gradients(outputs, inputs):
    return torch.autograd.grad(outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True)

# Convert torch tensor into np.array
def to_numpy(input):
    if isinstance(input, torch.Tensor):
        return input.detach().cpu().numpy()
    elif isinstance(input, np.ndarray):
        return input
    else:
        raise TypeError('Unknown type of input, expected torch.Tensor or ' \
                        'np.ndarray, but got {}'.format(type(input)))

# Initial conditions
def IC(x):
    N = len(x)
    u_init = np.zeros((x.shape[0]))
    for i in range(N):
        if (-0.5 <= x[i] and x[i] <= 0):
            u_init[i] = 1
        else:
            u_init[i] = 0
    return u_init

In [39]:
def train(epoch, model):
    model.train()
    def closure():
        optimizer.zero_grad()                                                     # Optimizer
        loss_pde = model.loss_pde(x_int_train)                                    # Loss function of PDE
        loss_ic = model.loss_ic(x_ic_train,u_ic_train)   # Loss function of IC
        loss = 0.1*loss_pde + 10*loss_ic                                          # Total loss function G(theta)

        # Print iteration, loss of PDE and ICs
        if epoch % 1000 == 0:
            print(f'epoch {epoch} loss_pde:{loss_pde:.8f}, loss_ic:{loss_ic:.8f}')

        loss.backward()
        return loss

    # Optimize loss function
    loss = optimizer.step(closure)
    loss_value = loss.item() if not isinstance(loss, float) else loss
    # Print total loss
    if epoch % 1000 == 0:
        print(f'epoch {epoch}: loss {loss_value:.6f}')
    return loss_value

In [14]:
# Initialization
device = torch.device('cpu')                                          # Run on CPU
lr = 0.0005                                                           # Learning rate
num_x = 1000                                                          # Number of points in t
num_t = 1000                                                          # Number of points in x
num_i_train = 1000                                                    # Random sampled points from IC
epochs = 10000                                                        # Number of iterations
num_f_train = 11000                                                   # Random sampled points in interior
x = np.linspace(-2.625, 2.5,num_x)                                    # Partitioned spatial axis
t = np.linspace(0, 1.5, num_t)                                        # Partitioned time axis
t_grid, x_grid = np.meshgrid(t, x)                                    # (t,x) in [0,0.2]x[a,b]
T = t_grid.flatten()[:, None]                                         # Vectorized t_grid
X = x_grid.flatten()[:, None]                                         # Vectorized x_grid

xs = np.linspace(-1, 1, num_x)                                         # Partitioned spatial axis
ts = np.linspace(0, 1.5, num_t)                                       # Partitioned time axis
ts_grid, xs_grid = np.meshgrid(ts, xs)                                # (t,x) in [0,0.2]x[a,b]
Ts = ts_grid.flatten()[:, None]                                       # Vectorized t_grid
Xs = xs_grid.flatten()[:, None]

id_ic = np.random.choice(num_x, num_i_train, replace=False)           # Random sample numbering for IC
id_f = np.random.choice(num_x*num_t, num_f_train, replace=False)      # Random sample numbering for interior

x_ic = x_grid[id_ic, 0][:, None]                                      # Random x - initial condition
t_ic = t_grid[id_ic, 0][:, None]                                      # random t - initial condition
x_ic_train = np.hstack((t_ic, x_ic))                                  # Random (x,t) - vectorized
u_ic_train = IC(x_ic)                       # Initial condition evaluated at random sample

x_int = X[:, 0][id_f, None]                                           # Random x - interior
t_int = T[:, 0][id_f, None]                                           # Random t - interior
x_int_train = np.hstack((t_int, x_int))                               # Random (x,t) - vectorized
x_test = np.hstack((Ts, Xs))                                          # Vectorized whole domai

In [15]:
# Generate tensors
x_ic_train = torch.tensor(x_ic_train, dtype=torch.float32).to(device)
x_int_train = torch.tensor(x_int_train, requires_grad=True, dtype=torch.float32).to(device)
x_test = torch.tensor(x_test, dtype=torch.float32).to(device)

u_ic_train = torch.tensor(u_ic_train, dtype=torch.float32).to(device)

In [16]:
%%time
# Initialize neural network
model = DNN().to(device)

# Loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

loss = []
for epoch in range(1, epochs+1):
    loss.append(train(epoch, model))

# Evaluate on the whole computational domain
u_pred = to_numpy(model(x_test))

epoch 1000 loss_pde:0.11446005, loss_ic:0.00162144
epoch 1000: loss 0.027660
epoch 2000 loss_pde:0.05151062, loss_ic:0.00061292
epoch 2000: loss 0.011280
epoch 3000 loss_pde:0.03873724, loss_ic:0.00023005
epoch 3000: loss 0.006174
epoch 4000 loss_pde:0.03429297, loss_ic:0.00007947
epoch 4000: loss 0.004224
epoch 5000 loss_pde:0.03291442, loss_ic:0.00005588
epoch 5000: loss 0.003850
epoch 6000 loss_pde:0.03219775, loss_ic:0.00003908
epoch 6000: loss 0.003611
epoch 7000 loss_pde:0.03064250, loss_ic:0.00002886
epoch 7000: loss 0.003353
epoch 8000 loss_pde:0.02899260, loss_ic:0.00002256
epoch 8000: loss 0.003125
epoch 9000 loss_pde:0.02785664, loss_ic:0.00015531
epoch 9000: loss 0.004339
epoch 10000 loss_pde:0.03751867, loss_ic:0.00038231
epoch 10000: loss 0.007575
CPU times: user 10min 3s, sys: 2.61 s, total: 10min 6s
Wall time: 10min 6s


In [17]:
data = pd.DataFrame(data={'t': [i[0] for i in x_test.numpy()], 
                          'x': [i[1] for i in x_test.numpy()], 
                          'u': u_pred.flatten()})

In [18]:
sample = data.sample(1000).sort_values('t')

In [19]:
sample.sample(10).sort_index()

,t,x,u
56954,1.432432,-0.887888,-0.043762
73816,1.225225,-0.853854,-0.040743
114168,0.252252,-0.771772,0.007677
453038,0.057057,-0.093093,0.981481
483499,0.749249,-0.033033,0.732623
540253,0.379880,0.081081,0.584699
609331,0.496997,0.219219,0.428126
796858,1.288288,0.593594,0.417783
854805,1.208709,0.709710,0.108795
875446,0.669670,0.751752,0.031863


In [20]:
fig = go.Figure(data=[go.Surface(x=sample['x'], y=sample['t'], z=sample['u'])])
# fig.update_layout(title='Non-convex flux without diffusion term')
fig.write_image('images/nonconvex_flux_without_diffusion_term_problem_2.png')

![](images/nonconvex_flux_without_diffusion_term_problem_2.png)

not sure why this isnt working, will try to get some slices from the data

In [21]:
def plot_ts(data, ts):
    fig = make_subplots(rows=1, cols=3)
    for i, t in enumerate(ts):
        # I'll use a +-0.01 range as equal to my desired timestamp
        start, end = t - 0.01, t + 0.01
        aux = data[(data['t'] > start) & (data['t'] < end)]
        aux_fig = go.Scatter(x=aux['x'], y=aux['u'], name=f'{t=}')
        fig.update_yaxes(range=[0, 1])
        fig.add_trace(aux_fig, row=1, col=i+1)

    fig.update_layout(height=600, width=1000)
    return fig

In [22]:
plot_ts(data, [0.25, 0.5, 0.75]).write_image('images/problem_2_solutions.png')

![](images/problem_2_solutions.png)

In [24]:
plot_ts(data, [0.5, 1, 1.5]).write_image('images/problem_2_solutions_2.png')

![](images/problem_2_solutions_2.png)

---

I'll use the same trained model here on the data generated in 1a, it should work

In [31]:
bl = pd.read_parquet('Data/buckley_leverett.parquet')
x = torch.tensor(bl[['x', 't']].to_numpy(), dtype=torch.float32).to(device)
bl['u_pred'] = model(x).detach().numpy()

In [32]:
bl.sample(10).sort_index()

,x,t,u,u_pred
2513,0.24,0.89,0.899427,0.010766
3143,0.31,0.12,0.000000,0.456223
3632,0.35,0.97,0.874368,0.014433
3869,0.38,0.31,0.000000,0.060304
4757,0.47,0.10,0.000000,0.585297
5576,0.55,0.21,0.000000,0.459942
6034,0.59,0.75,0.781681,0.029466
6949,0.68,0.81,0.771804,0.030574
8641,0.85,0.56,0.000000,0.083996
9917,0.98,0.19,0.000000,0.598906


In [33]:
def plot_ts_comp(df, t):
    aux = df[df['t'] == t]
    fig = go.Figure()
    fig.update_yaxes(range=[0, 1])
    fig.add_trace(go.Scatter(x=aux['x'], y=aux['u'], name='true'))
    fig.add_trace(go.Scatter(x=aux['x'], y=aux['u_pred'], name='pred'))
    return fig

In [34]:
plot_ts_comp(bl, 0.25).write_image('images/problem_2_solutions_3.png')

![](images/problem_2_solutions_3.png)

In [35]:
plot_ts_comp(bl, 0.5).write_image('images/problem_2_solutions_4.png')

![](images/problem_2_solutions_4.png)

In [36]:
plot_ts_comp(bl, 0.75).write_image('images/problem_2_solutions_5.png')

![](images/problem_2_solutions_5.png)

this is very very wrong, probably because of the discontinuity, will add the diffusive term

In [37]:
class BLWithDiffusion(DNN):
    # Loss function for PDE
    def loss_pde(self, x):
        u = self.net(x)
        # Gradients and partial derivatives
        du_g = gradients(u, x)[0]                                  # Gradient [u_t, u_x]
        u_t, u_x = du_g[:, :1], du_g[:, 1:]                        # Partial derivatives u_t, u_x
        F = 1e-3 + (u**2)/(4*(u**2) + (1-u)**2)
        DF = gradients(F, x)[0]                                    # Gradient of flux DF
        F_x = DF[:, 1:]                                            # Partial derivativEe of flux, F(u)_x

        # Loss function for the Euler Equations
        f = ((u_t + F_x)**2).mean()
        return f

In [40]:
%%time
epochs = 20000
model = BLWithDiffusion().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

loss = []
for epoch in range(1, epochs+1):
    loss.append(train(epoch, model))

u_pred = to_numpy(model(x_test))

epoch 1000 loss_pde:0.09838654, loss_ic:0.00172407
epoch 1000: loss 0.027079
epoch 2000 loss_pde:0.05089094, loss_ic:0.00060041
epoch 2000: loss 0.011093
epoch 3000 loss_pde:0.04008782, loss_ic:0.00016700
epoch 3000: loss 0.005679
epoch 4000 loss_pde:0.03461559, loss_ic:0.00035941
epoch 4000: loss 0.007056
epoch 5000 loss_pde:0.02638361, loss_ic:0.00005949
epoch 5000: loss 0.003233
epoch 6000 loss_pde:0.02241264, loss_ic:0.00006012
epoch 6000: loss 0.002842
epoch 7000 loss_pde:0.02257476, loss_ic:0.00002214
epoch 7000: loss 0.002479
epoch 8000 loss_pde:0.03521419, loss_ic:0.00040277
epoch 8000: loss 0.007549
epoch 9000 loss_pde:0.02473850, loss_ic:0.00013087
epoch 9000: loss 0.003783
epoch 10000 loss_pde:0.02164688, loss_ic:0.00003010
epoch 10000: loss 0.002466
epoch 11000 loss_pde:0.01715076, loss_ic:0.00000684
epoch 11000: loss 0.001783
epoch 12000 loss_pde:0.01574715, loss_ic:0.00001926
epoch 12000: loss 0.001767
epoch 13000 loss_pde:0.01414544, loss_ic:0.00000197
epoch 13000: loss 

In [47]:
px.line(x=range(len(loss)), y=loss, title='NN loss per epoch').write_image('images/pinn_bl_loss.png')

![](images/pinn_bl_loss.png)

something is working, or the loss wouldn't be getting smaller

In [48]:
bl = pd.read_parquet('Data/buckley_leverett.parquet')
x = torch.tensor(bl[['x', 't']].to_numpy(), dtype=torch.float32).to(device)
bl['u_pred'] = model(x).detach().numpy()

In [43]:
bl.sample(10).sort_index()

,x,t,u,u_pred
114,0.01,0.13,0.965314,-0.001125
1984,0.19,0.65,0.892931,0.000740
2016,0.19,0.97,0.922111,0.000690
3871,0.38,0.33,0.716651,0.050713
4501,0.44,0.57,0.784477,0.006850
4689,0.46,0.43,0.730757,0.037243
5308,0.52,0.56,0.755605,0.015162
6505,0.64,0.41,0.000000,0.087126
8240,0.81,0.59,0.000000,0.056311
9574,0.94,0.80,0.712617,0.029995


In [44]:
plot_ts_comp(bl, 0.25).write_image('images/problem_2_solutions_6.png')

![](images/problem_2_solutions_6.png)

In [45]:
plot_ts_comp(bl, 0.5).write_image('images/problem_2_solutions_7.png')

![](images/problem_2_solutions_7.png)

In [46]:
plot_ts_comp(bl, 0.75).write_image('images/problem_2_solutions_8.png')

![](images/problem_2_solutions_8.png)

same thing, something is wrong

---

I'll try a different approach, sklearn MLP optimized with Optuna, I'm more familiar with the libs

In [74]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna

In [55]:
bl = pd.read_parquet('Data/buckley_leverett.parquet').dropna()

In [59]:
X = bl[['x', 't']]
y = bl[['u']]

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [68]:
# same parameters as the ones used in torch before, for testing
model = MLPRegressor(hidden_layer_sizes=tuple([30] * 7), activation='tanh')

In [69]:
model.fit(X_train, y_train)

/Users/lucasabbade/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



MLPRegressor(activation='tanh', hidden_layer_sizes=(30, 30, 30, 30, 30, 30, 30))

In [71]:
y_pred = model.predict(X_test)

In [83]:
def rmse(y_test, y_pred):
    return mean_squared_error(y_test, y_pred) ** 0.5

In [84]:
f'RMSE = {rmse(y_test["u"], y_pred):.6f}'

'RMSE = 0.039578'

In [95]:
bl['u_pred'] = model.predict(X)

In [96]:
bl

,x,t,u,u_pred
1,0.0,0.01,1.000000,0.774899
2,0.0,0.02,1.000000,0.824647
3,0.0,0.03,1.000000,0.830117
4,0.0,0.04,1.000000,0.834763
5,0.0,0.05,1.000000,0.841274
...,...,...,...,...
10196,1.0,0.96,0.735642,0.740366
10197,1.0,0.97,0.737516,0.743553
10198,1.0,0.98,0.739355,0.746105
10199,1.0,0.99,0.741161,0.748100


In [97]:
plot_ts_comp(bl, 0.25).write_image('images/problem_2_solutions_9.png')

![](images/problem_2_solutions_9.png)

In [100]:
plot_ts_comp(bl, 0.5).write_image('images/problem_2_solutions_10.png')

![](images/problem_2_solutions_10.png)

In [101]:
plot_ts_comp(bl, 0.75).write_image('images/problem_2_solutions_11.png')

![](images/problem_2_solutions_11.png)

Now we're talking! I'm not familiar with pytorch, there's probably something wrong with the implementation I was using.